In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)


**Filter overall Average**

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

# Load data
deliveries = pd.read_csv('all_matches_2023_onwards.csv')
df = deliveries.copy()

# Data preparation (following your pattern)
df = df.rename(columns={'striker': 'batsman'})
df = df.rename(columns={'runs_off_bat': 'runs_of_bat'})
df['innings'] = df['innings'].astype(int)

# Fill missing values
df['wides'] = df['wides'].fillna(0)
df['noballs'] = df['noballs'].fillna(0)

# Calculate total runs
df['total_runs'] = df['runs_of_bat'] + df['wides'] + df['noballs']

# Create match-level results to determine winners
match_results = df.groupby(['venue', 'match_id', 'innings'])['total_runs'].sum().reset_index()
match_results_pivot = match_results.pivot(index=['venue', 'match_id'], columns='innings', values='total_runs').fillna(0)
match_results_pivot = match_results_pivot.rename(columns={1: 'innings_1_score', 2: 'innings_2_score'})
match_results_pivot = match_results_pivot.reset_index()

# Determine match results
match_results_pivot['result'] = match_results_pivot.apply(
    lambda x: 'runs' if x['innings_1_score'] > x['innings_2_score'] else 'wickets', axis=1
)

# Add target_runs column (innings 1 score + 1)
match_results_pivot['target_runs'] = match_results_pivot['innings_1_score'] + 1

# Venue Analysis (exactly like your reference format)
vdf_new = (
    df.groupby(['venue', 'innings'])['total_runs']
    .sum()
    .unstack(fill_value=0)
    .rename(columns={1: 1, 2: 2})
    .assign(
        total_match_run=lambda x: x.sum(axis=1),  # Total runs in the match
        match_count=lambda x: df.groupby('venue')['match_id'].nunique(),  # Total matches at the venue
        avg_innings_1=lambda x: x[1] / x['match_count'],  # Avg runs in innings 1
        avg_innings_2=lambda x: x[2] / x['match_count'],  # Avg runs in innings 2
        
        # Innings wins count (using match results)
        inning_1_wins=lambda x: match_results_pivot[match_results_pivot['result'] == "runs"]
            .groupby('venue')['match_id'].nunique().reindex(x.index, fill_value=0),
        inning_2_wins=lambda x: match_results_pivot[match_results_pivot['result'] == "wickets"]
            .groupby('venue')['match_id'].nunique().reindex(x.index, fill_value=0),
        
        # Highest Score (HS) - Maximum innings total at each venue
        HS=lambda x: df.groupby(['venue', 'match_id', 'innings'])['total_runs']
            .sum().groupby('venue').max().reindex(x.index, fill_value=0),
        
        # Lowest Score (LS) - Minimum innings total at each venue
        LS=lambda x: df.groupby(['venue', 'match_id', 'innings'])['total_runs']
            .sum().groupby('venue').min().reindex(x.index, fill_value=0),
        
        # Highest Chase (HC) - Maximum successful chase at each venue
        HC=lambda x: match_results_pivot[match_results_pivot['result'] == "wickets"]
            .groupby('venue')['innings_2_score'].max().reindex(x.index, fill_value=0),
        
        # Lowest Defended (LD) - Minimum successfully defended total at each venue
        LD=lambda x: match_results_pivot[match_results_pivot['result'] == "runs"]
            .groupby('venue')['innings_1_score'].min().reindex(x.index, fill_value=0),
    )
)

# Compute overall average runs per match
vdf_new['avg_runs'] = vdf_new[['avg_innings_1', 'avg_innings_2']].mean(axis=1)

# Reorder columns exactly as requested
vdf_new = vdf_new[[1, 2, 'total_match_run', 'match_count', 'avg_innings_1', 'avg_innings_2', 
                   'inning_1_wins', 'inning_2_wins', 'HS', 'LS', 'HC', 'LD', 'avg_runs']]

# Fill missing values with 0 (if any)
vdf_new = vdf_new.fillna(0)

# Round numerical columns
vdf_new = vdf_new.round({
    'avg_innings_1': 2,
    'avg_innings_2': 2,
    'avg_runs': 2
})

# Sort by average runs (descending)
vdf_new = vdf_new.sort_values('avg_runs', ascending=False)

# Display results
print("🏟️ COMPREHENSIVE VENUE ANALYSIS")
print("=" * 80)
vdf_new.head(25)


🏟️ COMPREHENSIVE VENUE ANALYSIS


innings,1,2,total_match_run,match_count,avg_innings_1,avg_innings_2,inning_1_wins,inning_2_wins,HS,LS,HC,LD,avg_runs
venue,,,,,,,,,,,,,
"Merchant Taylors' School Ground, Northwood",417.0,366.0,783.0,2,208.50,183.00,2,0,236.0,172.0,0.0,181.0,195.75
"County Ground, Chelmsford",2751.0,2531.0,5282.0,15,183.40,168.73,5,10,225.0,90.0,225.0,173.0,176.07
"County Ground, Hove",2740.0,2429.0,5169.0,15,182.67,161.93,9,6,257.0,111.0,203.0,155.0,172.30
"Stanley Park, Blackpool",169.0,175.0,344.0,1,169.00,175.00,0,1,175.0,169.0,175.0,0.0,172.00
"Headingley, Leeds",2463.0,2269.0,4732.0,14,175.93,162.07,9,5,222.0,96.0,196.0,164.0,169.00
"County Ground, Northampton",2437.0,2267.0,4704.0,14,174.07,161.93,9,5,207.0,93.0,207.0,139.0,168.00
"Sophia Gardens, Cardiff",2382.0,1983.0,4365.0,13,183.23,152.54,10,3,242.0,86.0,191.0,87.0,167.88
"Kennington Oval, London",2420.0,2213.0,4633.0,14,172.86,158.07,8,6,250.0,80.0,198.0,96.0,165.46
"County Ground, New Road, Worcester",2368.0,2174.0,4542.0,14,169.14,155.29,7,7,213.0,99.0,206.0,134.0,162.21


##**Filter with venue**

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

# Load data
deliveries = pd.read_csv('all_matches_2023_onwards.csv')
df = deliveries.copy()

# Data preparation (following your pattern)
df = df.rename(columns={'striker': 'batsman'})
df = df.rename(columns={'runs_off_bat': 'runs_of_bat'})
df['innings'] = df['innings'].astype(int)

# Fill missing values
df['wides'] = df['wides'].fillna(0)
df['noballs'] = df['noballs'].fillna(0)

# Calculate total runs
df['total_runs'] = df['runs_of_bat'] + df['wides'] + df['noballs']

# Create match-level results to determine winners
match_results = df.groupby(['venue', 'match_id', 'innings'])['total_runs'].sum().reset_index()
match_results_pivot = match_results.pivot(index=['venue', 'match_id'], columns='innings', values='total_runs').fillna(0)
match_results_pivot = match_results_pivot.rename(columns={1: 'innings_1_score', 2: 'innings_2_score'})
match_results_pivot = match_results_pivot.reset_index()

# Determine match results
match_results_pivot['result'] = match_results_pivot.apply(
    lambda x: 'runs' if x['innings_1_score'] > x['innings_2_score'] else 'wickets', axis=1
)

# Add target_runs column (innings 1 score + 1)
match_results_pivot['target_runs'] = match_results_pivot['innings_1_score'] + 1

# Venue Analysis (exactly like your reference format)
vdf_new = (
    df.groupby(['venue', 'innings'])['total_runs']
    .sum()
    .unstack(fill_value=0)
    .rename(columns={1: 1, 2: 2})
    .assign(
        total_match_run=lambda x: x.sum(axis=1),  # Total runs in the match
        match_count=lambda x: df.groupby('venue')['match_id'].nunique(),  # Total matches at the venue
        avg_innings_1=lambda x: x[1] / x['match_count'],  # Avg runs in innings 1
        avg_innings_2=lambda x: x[2] / x['match_count'],  # Avg runs in innings 2
        
        # Innings wins count (using match results)
        inning_1_wins=lambda x: match_results_pivot[match_results_pivot['result'] == "runs"]
            .groupby('venue')['match_id'].nunique().reindex(x.index, fill_value=0),
        inning_2_wins=lambda x: match_results_pivot[match_results_pivot['result'] == "wickets"]
            .groupby('venue')['match_id'].nunique().reindex(x.index, fill_value=0),
        
        # Highest Score (HS) - Maximum innings total at each venue
        HS=lambda x: df.groupby(['venue', 'match_id', 'innings'])['total_runs']
            .sum().groupby('venue').max().reindex(x.index, fill_value=0),
        
        # Lowest Score (LS) - Minimum innings total at each venue
        LS=lambda x: df.groupby(['venue', 'match_id', 'innings'])['total_runs']
            .sum().groupby('venue').min().reindex(x.index, fill_value=0),
        
        # Highest Chase (HC) - Maximum successful chase at each venue
        HC=lambda x: match_results_pivot[match_results_pivot['result'] == "wickets"]
            .groupby('venue')['innings_2_score'].max().reindex(x.index, fill_value=0),
        
        # Lowest Defended (LD) - Minimum successfully defended total at each venue
        LD=lambda x: match_results_pivot[match_results_pivot['result'] == "runs"]
            .groupby('venue')['innings_1_score'].min().reindex(x.index, fill_value=0),
    )
)

# Compute overall average runs per match
vdf_new['avg_runs'] = vdf_new[['avg_innings_1', 'avg_innings_2']].mean(axis=1)

# Reorder columns exactly as requested
vdf_new = vdf_new[[1, 2, 'total_match_run', 'match_count', 'avg_innings_1', 'avg_innings_2', 
                   'inning_1_wins', 'inning_2_wins', 'HS', 'LS', 'HC', 'LD', 'avg_runs']]

# Fill missing values with 0 (if any)
vdf_new = vdf_new.fillna(0)

# Round numerical columns
vdf_new = vdf_new.round({
    'avg_innings_1': 2,
    'avg_innings_2': 2,
    'avg_runs': 2
})

# Sort by average runs (descending)
vdf_new = vdf_new.sort_values('avg_runs', ascending=False)

# Enhanced function to get last 10 matches with detailed scores and winner
def get_last_10_matches_with_detailed_scores(df, venue_name):
    """Get last 10 matches with team names, detailed scores, and match results"""
    
    # Filter data for the specific venue
    venue_df = df[df['venue'] == venue_name].copy()
    
    if venue_df.empty:
        print(f"No data found for venue: {venue_name}")
        return None
    
    # Get match totals with team names for each innings
    match_details = venue_df.groupby(['match_id', 'innings', 'batting_team'])['total_runs'].sum().reset_index()
    
    # Get wickets data for each innings
    wickets_data = venue_df.groupby(['match_id', 'innings', 'batting_team'])['player_dismissed'].count().reset_index()
    wickets_data = wickets_data.rename(columns={'player_dismissed': 'wickets'})
    
    # Merge runs and wickets
    match_details = pd.merge(match_details, wickets_data, on=['match_id', 'innings', 'batting_team'], how='left')
    match_details['wickets'] = match_details['wickets'].fillna(0)
    
    # Separate innings 1 and 2
    inn1_details = match_details[match_details['innings'] == 1][['match_id', 'batting_team', 'total_runs', 'wickets']].rename(
        columns={'batting_team': 'team_1', 'total_runs': 'score_1', 'wickets': 'wickets_1'})
    
    inn2_details = match_details[match_details['innings'] == 2][['match_id', 'batting_team', 'total_runs', 'wickets']].rename(
        columns={'batting_team': 'team_2', 'total_runs': 'score_2', 'wickets': 'wickets_2'})
    
    # Merge both innings
    complete_matches = pd.merge(inn1_details, inn2_details, on='match_id', how='inner')
    
    # Determine winner and margin
    def get_match_result(row):
        if row['score_1'] > row['score_2']:
            margin = row['score_1'] - row['score_2']
            return f"{row['team_1']} won by {margin} runs"
        elif row['score_2'] > row['score_1']:
            wickets_remaining = 10 - row['wickets_2']
            return f"{row['team_2']} won by {wickets_remaining} wickets"
        else:
            return "Match Drawn"
    
    complete_matches['result'] = complete_matches.apply(get_match_result, axis=1)
    
    # Create detailed team vs team format with scores
    def create_teams_vs_format(row):
        team1_score = f"{row['team_1']} {int(row['score_1'])}-{int(row['wickets_1'])}"
        team2_score = f"{row['team_2']} {int(row['score_2'])}-{int(row['wickets_2'])}"
        return f"{team1_score} vs {team2_score}"
    
    complete_matches['teams_vs'] = complete_matches.apply(create_teams_vs_format, axis=1)
    
    # Sort by match_id descending (most recent first)
    complete_matches = complete_matches.sort_values('match_id', ascending=False)
    
    # Take last 10 matches
    last_10_matches = complete_matches.head(10).copy()
    
    # Create final display DataFrame
    matches_display = last_10_matches[['match_id', 'teams_vs', 'result']].reset_index(drop=True)
    matches_display.index += 1
    matches_display.index.name = 'Match No.'
    matches_display = matches_display.rename(columns={
        'match_id': 'Match ID',
        'teams_vs': 'Teams',
        'result': 'Result'
    })
    
    # Display results
    from IPython.display import display
    
    print(f"🏟️ VENUE ANALYSIS: {venue_name}")
    print("=" * 80)
    print("\n📅 LAST 10 MATCHES:")
    
    # Display as clean table
    display(matches_display)
    
    # Get venue stats
    if venue_name in vdf_new.index:
        venue_stats = vdf_new.loc[[venue_name]]
        print(f"\n📊 VENUE STATISTICS:")
        display(venue_stats)
    else:
        print(f"\nVenue statistics not found for: {venue_name}")
    
    return {
        'last_10_matches': matches_display,
        'venue_stats': venue_stats if venue_name in vdf_new.index else None
    }

# Example usage
venue_name = 'Edgbaston, Birmingham'
result = get_last_10_matches_with_detailed_scores(df, venue_name)


🏟️ VENUE ANALYSIS: Edgbaston, Birmingham

📅 LAST 10 MATCHES:


,Match ID,Teams,Result
Match No.,,,
1,1410499,Gloucestershire 133-10 vs Birmingham Bears 122-9,Gloucestershire won by 11.0 runs
2,1410494,Birmingham Bears 188-5 vs Leicestershire 121-10,Birmingham Bears won by 67.0 runs
3,1410479,Birmingham Bears 193-2 vs Derbyshire 152-10,Birmingham Bears won by 41.0 runs
4,1410472,Worcestershire 186-6 vs Birmingham Bears 132-10,Worcestershire won by 54.0 runs
5,1410432,Birmingham Bears 217-4 vs Northamptonshire 127-9,Birmingham Bears won by 90.0 runs
6,1410418,Yorkshire 142-10 vs Birmingham Bears 150-6,Birmingham Bears won by 4 wickets
7,1410401,Durham 191-9 vs Birmingham Bears 167-10,Durham won by 24.0 runs
8,1410384,Leicestershire 174-6 vs Derbyshire 170-6,Leicestershire won by 4.0 runs
9,1410383,Birmingham Bears 148-10 vs Nottinghamshire 126-10,Birmingham Bears won by 22.0 runs



📊 VENUE STATISTICS:


innings,1,2,total_match_run,match_count,avg_innings_1,avg_innings_2,inning_1_wins,inning_2_wins,HS,LS,HC,LD,avg_runs
venue,,,,,,,,,,,,,
"Edgbaston, Birmingham",3575.0,3051.0,6626.0,21,170.24,145.29,16,5,228.0,97.0,204.0,133.0,157.76


In [6]:
df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'batsman', 'non_striker', 'bowler',
       'runs_of_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'total_runs'],
      dtype='object')

In [7]:
df.batting_team.unique()

array(['Birmingham Bears', 'Yorkshire', 'Derbyshire', 'Lancashire',
       'Hampshire', 'Somerset', 'Worcestershire', 'Northamptonshire',
       'Gloucestershire', 'Kent', 'Leicestershire', 'Surrey', 'Middlesex',
       'Sussex', 'Durham', 'Glamorgan', 'Nottinghamshire', 'Essex'],
      dtype=object)

In [8]:
df.venue.unique()

array(['Edgbaston, Birmingham',
       'The Cooper Associates County Ground, Taunton',
       'County Ground, Northampton', 'St Lawrence Ground, Canterbury',
       'Old Trafford, Manchester', "Lord's, London",
       'Kennington Oval, London', 'County Ground, Hove',
       'Grace Road, Leicester', 'The Rose Bowl, Southampton',
       'County Ground, Bristol', 'Trent Bridge, Nottingham',
       'County Ground, New Road, Worcester', 'Headingley, Leeds',
       'County Ground, Derby',
       "Merchant Taylors' School Ground, Northwood",
       'Riverside Ground, Chester-le-Street', 'County Ground, Chelmsford',
       'Sophia Gardens, Cardiff', 'Radlett Cricket Club, Radlett',
       'Stanley Park, Blackpool', "Queen's Park, Chesterfield",
       'College Ground, Cheltenham'], dtype=object)

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

# Load data
deliveries = pd.read_csv('all_matches_updated.csv')
df = deliveries.copy()

# Data preparation (following your pattern)
df = df.rename(columns={'striker': 'batsman'})
df = df.rename(columns={'runs_off_bat': 'runs_of_bat'})
df['innings'] = df['innings'].astype(int)

# Fill missing values
df['wides'] = df['wides'].fillna(0)
df['noballs'] = df['noballs'].fillna(0)
#extras
# Calculate total runs
#df['total_runs'] = df['runs_of_bat'] + df['wides'] + df['noballs']
df['total_runs'] = df['runs_of_bat'] + df['extras']
# Function to check available teams
def check_available_teams():
    """Check what teams are available in the dataset"""
    all_teams = set(df['batting_team'].unique()) | set(df['bowling_team'].unique())
    print("📋 AVAILABLE TEAMS IN DATASET:")
    print("-" * 50)
    for i, team in enumerate(sorted(all_teams), 1):
        print(f"{i}. {team}")
    return sorted(all_teams)

# Function with debugging
def get_matches_between_teams_debug(df, team1, team2):
    """Get matches between teams with debugging information"""
    
    print(f"🔍 DEBUGGING: Looking for matches between '{team1}' and '{team2}'")
    
    # Check if teams exist in dataset
    all_batting_teams = set(df['batting_team'].unique())
    all_bowling_teams = set(df['bowling_team'].unique())
    all_teams = all_batting_teams | all_bowling_teams
    
    if team1 not in all_teams:
        print(f"❌ Team '{team1}' not found in dataset!")
        print("Available teams:")
        for team in sorted(all_teams):
            if team1.lower() in team.lower():
                print(f"   - {team} (similar to {team1})")
        return None
    
    if team2 not in all_teams:
        print(f"❌ Team '{team2}' not found in dataset!")
        print("Available teams:")
        for team in sorted(all_teams):
            if team2.lower() in team.lower():
                print(f"   - {team} (similar to {team2})")
        return None
    
    print(f"✅ Both teams found in dataset")
    
    # Filter matches where both teams played
    matches_team1 = df[(df['batting_team'] == team1) | (df['bowling_team'] == team1)]
    matches_team2 = df[(df['batting_team'] == team2) | (df['bowling_team'] == team2)]
    
    print(f"📊 {team1} played in {matches_team1['match_id'].nunique()} matches")
    print(f"📊 {team2} played in {matches_team2['match_id'].nunique()} matches")
    
    # Get match ids where both teams played
    match_ids_team1 = set(matches_team1['match_id'].unique())
    match_ids_team2 = set(matches_team2['match_id'].unique())
    common_match_ids = match_ids_team1.intersection(match_ids_team2)
    
    print(f"🎯 Common matches between both teams: {len(common_match_ids)}")
    
    if not common_match_ids:
        print(f"❌ No matches found between {team1} and {team2}")
        return None
    
    # Continue with the rest of the analysis
    matches_df = df[df['match_id'].isin(common_match_ids)].copy()
    
    # Calculate match totals
    match_totals = matches_df.groupby(['match_id', 'venue', 'innings', 'batting_team']).agg({
        'total_runs': 'sum',
        'start_date': 'first'
    }).reset_index()
    
    print(f"📈 Processing {len(match_totals)} innings from {len(common_match_ids)} matches")
    
    # Count actual wickets
    wickets_per_innings = matches_df[matches_df['player_dismissed'].notna()].groupby(['match_id', 'innings', 'batting_team']).size().reset_index(name='wickets')
    
    # Merge runs and wickets
    match_details = pd.merge(match_totals, wickets_per_innings, on=['match_id', 'innings', 'batting_team'], how='left')
    match_details['wickets'] = match_details['wickets'].fillna(0)
    
    # Separate innings 1 and 2
    inn1_details = match_details[match_details['innings'] == 1][['match_id', 'venue', 'start_date', 'batting_team', 'total_runs', 'wickets']].rename(
        columns={'batting_team': 'team_1', 'total_runs': 'score_1', 'wickets': 'wickets_1'})
    
    inn2_details = match_details[match_details['innings'] == 2][['match_id', 'venue', 'batting_team', 'total_runs', 'wickets']].rename(
        columns={'batting_team': 'team_2', 'total_runs': 'score_2', 'wickets': 'wickets_2'})
    
    # Merge both innings
    complete_matches = pd.merge(inn1_details, inn2_details, on=['match_id', 'venue'], how='inner')
    
    print(f"🔄 Complete matches with both innings: {len(complete_matches)}")
    
    # Filter matches where teams are exactly team1 and team2
    cond1 = (complete_matches['team_1'] == team1) & (complete_matches['team_2'] == team2)
    cond2 = (complete_matches['team_1'] == team2) & (complete_matches['team_2'] == team1)
    complete_matches = complete_matches[cond1 | cond2].copy()
    
    print(f"✅ Final head-to-head matches: {len(complete_matches)}")
    
    if complete_matches.empty:
        print(f"❌ No direct head-to-head matches found between {team1} and {team2}")
        print("This might happen if they played in different groups/tournaments")
        return None
    
    # Calculate venue averages
    all_innings_totals = df.groupby(['venue', 'match_id', 'innings'])['total_runs'].sum().reset_index()
    venue_overall_avg = all_innings_totals.groupby('venue')['total_runs'].mean()
    
    team1_data = df[df['batting_team'] == team1]
    team1_innings = team1_data.groupby(['venue', 'match_id', 'innings'])['total_runs'].sum().reset_index()
    team1_venue_avg = team1_innings.groupby('venue')['total_runs'].mean()
    
    team2_data = df[df['batting_team'] == team2]
    team2_innings = team2_data.groupby(['venue', 'match_id', 'innings'])['total_runs'].sum().reset_index()
    team2_venue_avg = team2_innings.groupby('venue')['total_runs'].mean()
    
    # Get venues where these teams played against each other
    venues_played = complete_matches['venue'].unique()
    
    # Create venue stats
    venue_stats_list = []
    for venue in venues_played:
        venue_stats_list.append({
            'Venue': venue,
            'Venue Overall Avg': round(venue_overall_avg.get(venue, 0), 2),
            f'{team1} Avg': round(team1_venue_avg.get(venue, 0), 2),
            f'{team2} Avg': round(team2_venue_avg.get(venue, 0), 2),
            'Head-to-Head Matches': len(complete_matches[complete_matches['venue'] == venue])
        })
    
    venue_stats_df = pd.DataFrame(venue_stats_list)
    venue_stats_df.index += 1
    venue_stats_df.index.name = 'S.No.'
    
    # Create match results
    def get_match_result(row):
        score_1 = int(row['score_1'])
        score_2 = int(row['score_2'])
        wickets_2 = int(row['wickets_2'])
        
        if score_1 > score_2:
            margin = score_1 - score_2
            return f"{row['team_1']} won by {margin} runs"
        elif score_2 > score_1:
            wickets_remaining = 10 - wickets_2
            return f"{row['team_2']} won by {wickets_remaining} wickets"
        else:
            return "Match Drawn"
    
    complete_matches['result'] = complete_matches.apply(get_match_result, axis=1)
    
    def create_teams_vs_format(row):
        score_1 = int(row['score_1'])
        score_2 = int(row['score_2'])
        wickets_1 = int(row['wickets_1'])
        wickets_2 = int(row['wickets_2'])
        
        team1_score = f"{row['team_1']} {score_1}-{wickets_1}"
        team2_score = f"{row['team_2']} {score_2}-{wickets_2}"
        return f"{team1_score} vs {team2_score}"
    
    complete_matches['teams_vs'] = complete_matches.apply(create_teams_vs_format, axis=1)
    
    # Sort by match_id descending
    complete_matches = complete_matches.sort_values('match_id', ascending=False)
    
    # Create final display DataFrame
    if 'start_date' in complete_matches.columns:
        output_df = complete_matches[['match_id', 'venue', 'start_date', 'teams_vs', 'result']].reset_index(drop=True)
        output_df = output_df.rename(columns={
            'match_id': 'Match ID',
            'venue': 'Venue',
            'start_date': 'Date',
            'teams_vs': 'Teams',
            'result': 'Result'
        })
    else:
        output_df = complete_matches[['match_id', 'venue', 'teams_vs', 'result']].reset_index(drop=True)
        output_df = output_df.rename(columns={
            'match_id': 'Match ID',
            'venue': 'Venue',
            'teams_vs': 'Teams',
            'result': 'Result'
        })
    
    output_df.index += 1
    output_df.index.name = 'Match No.'
    
    # Calculate statistics
    team1_wins = complete_matches[complete_matches['result'].str.contains(team1, na=False)].shape[0]
    team2_wins = complete_matches[complete_matches['result'].str.contains(team2, na=False)].shape[0]
    draws = complete_matches[complete_matches['result'].str.contains('Drawn', na=False)].shape[0]
    
    venue_breakdown = complete_matches['venue'].value_counts()
    
    # Display results
    from IPython.display import display
    
    print(f"\n🏏 HEAD-TO-HEAD ANALYSIS: {team1} vs {team2}")
    print("=" * 80)
    
    print(f"\n📊 OVERALL STATISTICS:")
    print(f"Total Matches: {len(complete_matches)}")
    print(f"{team1} Wins: {team1_wins}")
    print(f"{team2} Wins: {team2_wins}")
    if draws > 0:
        print(f"Draws: {draws}")
    print(f"{team1} Win %: {(team1_wins/len(complete_matches)*100):.1f}%")
    print(f"{team2} Win %: {(team2_wins/len(complete_matches)*100):.1f}%")
    
    print(f"\n🏟️ VENUES WHERE THEY PLAYED:")
    for venue, count in venue_breakdown.items():
        print(f"• {venue}: {count} match(es)")
    
    print(f"\n📈 VENUE AVERAGES:")
    display(venue_stats_df)
    
    print(f"\n📅 ALL MATCHES:")
    display(output_df)
    
    return output_df

# First, check available teams
available_teams = check_available_teams()

# Try with exact team names from your dataset
# Replace these with actual team names from the list above
team1 = "Leicestershire"  # Replace with exact team name
team2 = "Derbyshire"  # Replace with exact team name

print(f"\n🎯 Attempting analysis with: '{team1}' vs '{team2}'")
result = get_matches_between_teams_debug(df, team1, team2)


📋 AVAILABLE TEAMS IN DATASET:
--------------------------------------------------
1. Birmingham Bears
2. Derbyshire
3. Durham
4. Essex
5. Glamorgan
6. Gloucestershire
7. Hampshire
8. Kent
9. Lancashire
10. Leicestershire
11. Middlesex
12. Northamptonshire
13. Nottinghamshire
14. Somerset
15. Surrey
16. Sussex
17. Warwickshire
18. Worcestershire
19. Yorkshire

🎯 Attempting analysis with: 'Leicestershire' vs 'Derbyshire'
🔍 DEBUGGING: Looking for matches between 'Leicestershire' and 'Derbyshire'
✅ Both teams found in dataset
📊 Leicestershire played in 148 matches
📊 Derbyshire played in 144 matches
🎯 Common matches between both teams: 22
📈 Processing 43 innings from 22 matches
🔄 Complete matches with both innings: 21
✅ Final head-to-head matches: 21

🏏 HEAD-TO-HEAD ANALYSIS: Leicestershire vs Derbyshire

📊 OVERALL STATISTICS:
Total Matches: 21
Leicestershire Wins: 8
Derbyshire Wins: 13
Leicestershire Win %: 38.1%
Derbyshire Win %: 61.9%

🏟️ VENUES WHERE THEY PLAYED:
• Grace Road: 7 match(es

,Venue,Venue Overall Avg,Leicestershire Avg,Derbyshire Avg,Head-to-Head Matches
S.No.,,,,,
1,Grace Road,151.92,150.26,150.71,7
2,Queen's Park,155.57,144.00,151.14,1
3,County Ground,161.13,149.60,158.08,4
4,"Grace Road, Leicester",163.73,159.57,183.00,5
5,"County Ground, Derby",168.70,171.67,170.45,3
6,"Edgbaston, Birmingham",154.17,150.00,174.67,1



📅 ALL MATCHES:


,Match ID,Venue,Date,Teams,Result
Match No.,,,,,
1,1460758,"Grace Road, Leicester",2025-05-30,Derbyshire 170-6 vs Leicestershire 171-5,Leicestershire won by 5 wickets
2,1410450,"Grace Road, Leicester",2024-07-06,Leicestershire 184-6 vs Derbyshire 188-6,Derbyshire won by 4 wickets
3,1410384,"Edgbaston, Birmingham",2024-06-01,Leicestershire 176-6 vs Derbyshire 178-6,Derbyshire won by 4 wickets
4,1347682,"County Ground, Derby",2023-06-30,Derbyshire 231-4 vs Leicestershire 160-9,Derbyshire won by 71 runs
5,1347600,"Grace Road, Leicester",2023-06-01,Derbyshire 189-5 vs Leicestershire 187-5,Derbyshire won by 2 runs
6,1297856,"County Ground, Derby",2022-06-09,Leicestershire 181-6 vs Derbyshire 182-4,Derbyshire won by 6 wickets
7,1297808,"Grace Road, Leicester",2022-05-28,Derbyshire 159-5 vs Leicestershire 89-10,Derbyshire won by 70 runs
8,1250292,"County Ground, Derby",2021-06-22,Leicestershire 174-8 vs Derbyshire 132-10,Leicestershire won by 42 runs
9,1250251,"Grace Road, Leicester",2021-06-11,Derbyshire 209-6 vs Leicestershire 186-10,Derbyshire won by 23 runs


In [10]:
df.batting_team.unique()

array(['Kent', 'Hampshire', 'Derbyshire', 'Lancashire', 'Worcestershire',
       'Nottinghamshire', 'Somerset', 'Essex', 'Gloucestershire',
       'Glamorgan', 'Birmingham Bears', 'Yorkshire', 'Leicestershire',
       'Surrey', 'Middlesex', 'Sussex', 'Durham', 'Northamptonshire'],
      dtype=object)

In [7]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

# Load data
deliveries = pd.read_csv('all_matches_2023_onwards.csv')
df = deliveries.copy()

# Data preparation (following your pattern)
df = df.rename(columns={'striker': 'batsman'})
df = df.rename(columns={'runs_off_bat': 'runs_of_bat'})
df['innings'] = df['innings'].astype(int)

# Fill missing values
df['wides'] = df['wides'].fillna(0)
df['noballs'] = df['noballs'].fillna(0)

# Calculate total runs
#df['total_runs'] = df['runs_of_bat'] + df['wides'] + df['noballs']
df['total_runs'] = df['runs_of_bat'] + df['extras']
def analyze_team_performance_at_venue(df, venue_name, team_name):
    """Analyze individual team performance at a specific venue"""
    
    # Filter dataset for matches played at the given venue
    venue_matches = df[df['venue'] == venue_name]

    # Filter for matches where the given team was the batting team
    team_matches_venue = venue_matches[venue_matches['batting_team'] == team_name]

    if team_matches_venue.empty:
        return None

    # Extract unique seasons (years) from the dataset if available
    if 'season' in df.columns:
        seasons = sorted(team_matches_venue['season'].unique())
    else:
        seasons = ["Data not available"]

    # Count the number of matches the team played at the venue
    team_match_count = team_matches_venue['match_id'].nunique()

    # Compute total runs per innings for the team
    team_innings_stats = (
        team_matches_venue.groupby(['match_id', 'innings'])['total_runs']
        .sum()
        .unstack(fill_value=0)
    )

    # FIXED: Added proper Series handling with dtype specification
    team_total_innings_1 = team_innings_stats.get(1, pd.Series(dtype=float)).sum()
    team_total_innings_2 = team_innings_stats.get(2, pd.Series(dtype=float)).sum()

    # Count how many times the team batted first or second
    team_bat_1st_count = team_innings_stats.get(1, pd.Series(dtype=float)).astype(bool).sum()
    team_bat_2nd_count = team_innings_stats.get(2, pd.Series(dtype=float)).astype(bool).sum()

    # Compute average runs per innings
    team_avg_innings_1 = team_total_innings_1 / team_bat_1st_count if team_bat_1st_count > 0 else 0
    team_avg_innings_2 = team_total_innings_2 / team_bat_2nd_count if team_bat_2nd_count > 0 else 0
    team_total_runs = team_total_innings_1 + team_total_innings_2

    # Compute Highest & Lowest Score (HS & LS)
    if not team_innings_stats.empty:
        team_HS = team_innings_stats.max().max()
        team_LS = team_innings_stats.replace(0, np.inf).min().min()
        if team_LS == np.inf:
            team_LS = 0
    else:
        team_HS = 0
        team_LS = 0

    # Calculate wins and determine HC/LD based on match results
    team_match_results = []
    
    for match_id in team_matches_venue['match_id'].unique():
        match_data = venue_matches[venue_matches['match_id'] == match_id]
        
        # Get innings totals for this match
        innings_totals = match_data.groupby(['innings', 'batting_team'])['total_runs'].sum().reset_index()
        
        if len(innings_totals) >= 2:
            inn1_data = innings_totals[innings_totals['innings'] == 1]
            inn2_data = innings_totals[innings_totals['innings'] == 2]
            
            if not inn1_data.empty and not inn2_data.empty:
                inn1_score = inn1_data['total_runs'].iloc[0]
                inn2_score = inn2_data['total_runs'].iloc[0]
                inn1_team = inn1_data['batting_team'].iloc[0]
                inn2_team = inn2_data['batting_team'].iloc[0]
                
                # Determine winner
                if inn1_score > inn2_score:
                    winner = inn1_team
                    result_type = "runs"
                else:
                    winner = inn2_team
                    result_type = "wickets"
                
                # Check if our team was involved and won
                if team_name == inn1_team:
                    team_score = inn1_score
                    team_innings = 1
                    team_won = (winner == team_name)
                elif team_name == inn2_team:
                    team_score = inn2_score
                    team_innings = 2
                    team_won = (winner == team_name)
                else:
                    continue
                
                team_match_results.append({
                    'match_id': match_id,
                    'team_score': team_score,
                    'team_innings': team_innings,
                    'team_won': team_won,
                    'result_type': result_type
                })

    # Calculate HC and LD
    team_HC = "N/A"
    team_LD = "N/A"
    
    if team_match_results:
        # Highest Chase (when team batted 2nd and won)
        successful_chases = [r['team_score'] for r in team_match_results 
                           if r['team_innings'] == 2 and r['team_won']]
        if successful_chases:
            team_HC = max(successful_chases)
        
        # Lowest Defended (when team batted 1st and won)
        successful_defenses = [r['team_score'] for r in team_match_results 
                             if r['team_innings'] == 1 and r['team_won']]
        if successful_defenses:
            team_LD = min(successful_defenses)

    # Calculate wins when batting first and second
    team_1st_bat_wins = len([r for r in team_match_results 
                           if r['team_innings'] == 1 and r['team_won']])
    team_2nd_bat_wins = len([r for r in team_match_results 
                           if r['team_innings'] == 2 and r['team_won']])

    # Calculate overall average
    team_overall_avg = team_total_runs / (team_bat_1st_count + team_bat_2nd_count) if (team_bat_1st_count + team_bat_2nd_count) > 0 else 0

    # Calculate win percentages
    team_1st_bat_win_percentage = (team_1st_bat_wins / team_bat_1st_count) * 100 if team_bat_1st_count > 0 else 0
    team_2nd_bat_win_percentage = (team_2nd_bat_wins / team_bat_2nd_count) * 100 if team_bat_2nd_count > 0 else 0

    # Display Results
    print(f"\n🏟️ Venue: {venue_name}")
    print(f"🏏 Team: {team_name}")
    if isinstance(seasons[0], str):
        print(f"🗓️ Seasons: {seasons[0]}")
    else:
        print(f"🗓️ Seasons: {', '.join(map(str, seasons))}")
    print(f"🟣 Matches played by {team_name}: {team_match_count}")
    
    print(f"🔥 Total Runs (1st Innings): {team_total_innings_1} || Avg: {team_avg_innings_1:.2f}")
    print(f"🔥 Total Runs (2nd Innings): {team_total_innings_2} || Avg: {team_avg_innings_2:.2f}")
    
    print(f"🎯 Total Runs (Overall): {team_total_runs} || Avg: {team_overall_avg:.2f}")
    
    print(f"🔵 {team_name} Batted 1st Count: {team_bat_1st_count}")
    print(f"🟢 {team_name} Batted 2nd Count: {team_bat_2nd_count}")
    
    print(f"🏆 Wins when Batting 1st: {team_1st_bat_wins} | Win %: {team_1st_bat_win_percentage:.2f}%")
    print(f"🏆 Wins when Batting 2nd: {team_2nd_bat_wins} | Win %: {team_2nd_bat_win_percentage:.2f}%")
    
    print(f"🚀 Highest Score (HS): {team_HS}")
    print(f"📉 Lowest Score (LS): {team_LS}")
    
    print(f"✅ Highest Chase (HC): {team_HC}")
    print(f"⚠️ Lowest Defended (LD): {team_LD}")
    
    # Return data for summary table
    return {
        'team': team_name,
        'matches': team_match_count,
        'overall_avg': round(team_overall_avg, 2),
        'win_pct_1st': round(team_1st_bat_win_percentage, 2),
        'win_pct_2nd': round(team_2nd_bat_win_percentage, 2),
        'HS': team_HS,
        'LS': team_LS
    }

def analyze_all_teams_at_venue(df, venue_name):
    """Analyze all teams' performance at a specific venue with summary table"""
    
    # Get all unique teams at the venue with their match counts
    teams_with_counts = df[df['venue'] == venue_name].groupby('batting_team')['match_id'].nunique().reset_index()
    teams_with_counts = teams_with_counts.rename(columns={'match_id': 'match_count'})
    
    # Sort teams by match count (descending) - teams with most matches first
    teams_with_counts = teams_with_counts.sort_values('match_count', ascending=False)
    
    if len(teams_with_counts) == 0:
        print(f"❌ No teams found at venue: {venue_name}")
        return
    
    print(f"🏟️ COMPREHENSIVE TEAM ANALYSIS AT {venue_name.upper()}")
    print("=" * 80)
    
    all_team_stats = []
    
    # Loop through teams sorted by match count (most matches first)
    for _, row in teams_with_counts.iterrows():
        team = row['batting_team']
        team_stats = analyze_team_performance_at_venue(df, venue_name, team)
        if team_stats:
            all_team_stats.append(team_stats)
        print("-" * 60)
    
    # Create and display summary table (also sorted by matches)
    if all_team_stats:
        summary_df = pd.DataFrame(all_team_stats)
        # Sort by matches first (descending), then by overall_avg (descending)
        summary_df = summary_df.sort_values(['matches', 'overall_avg'], ascending=[False, False])
        summary_df.index = range(len(summary_df))
        
        print(f"\n📊 TEAM PERFORMANCE SUMMARY AT {venue_name.upper()}")
        print("=" * 80)
        
        from IPython.display import display
        display(summary_df)

def analyze_team_at_all_venues(df, team_name):
    """Analyze a specific team's performance at all venues with summary table"""
    
    # Get all venues where this team played with match counts
    venues_with_counts = df[df['batting_team'] == team_name].groupby('venue')['match_id'].nunique().reset_index()
    venues_with_counts = venues_with_counts.rename(columns={'match_id': 'match_count'})
    
    # Sort venues by match count (descending) - venues with most matches first
    venues_with_counts = venues_with_counts.sort_values('match_count', ascending=False)
    
    if len(venues_with_counts) == 0:
        print(f"❌ No venues found for team: {team_name}")
        return
    
    print(f"🏏 COMPREHENSIVE VENUE ANALYSIS FOR {team_name.upper()}")
    print("=" * 80)
    
    all_venue_stats = []
    
    # Loop through venues sorted by match count (most matches first)
    for _, row in venues_with_counts.iterrows():
        venue = row['venue']
        venue_stats = analyze_team_performance_at_venue(df, venue, team_name)
        if venue_stats:
            venue_stats['venue'] = venue  # Add venue name to the stats
            all_venue_stats.append(venue_stats)
        print("-" * 60)
    
    # Create and display summary table (also sorted by matches)
    if all_venue_stats:
        summary_df = pd.DataFrame(all_venue_stats)
        # Sort by matches first (descending), then by overall_avg (descending)
        summary_df = summary_df.sort_values(['matches', 'overall_avg'], ascending=[False, False])
        summary_df.index = range(len(summary_df))
        
        # Reorder columns to show venue first
        cols = ['venue'] + [col for col in summary_df.columns if col != 'venue']
        summary_df = summary_df[cols]
        
        print(f"\n📊 {team_name.upper()} PERFORMANCE SUMMARY ACROSS ALL VENUES")
        print("=" * 80)
        
        from IPython.display import display
        display(summary_df)

def show_available_options(df):
    """Show available venues and teams for analysis"""
    
    venues = sorted(df['venue'].unique())
    teams = sorted(df['batting_team'].unique())
    
    print("🏟️ AVAILABLE VENUES:")
    print("-" * 30)
    for i, venue in enumerate(venues, 1):
        print(f"{i}. {venue}")
    
    print(f"\n🏏 AVAILABLE TEAMS:")
    print("-" * 30)
    for i, team in enumerate(teams, 1):
        print(f"{i}. {team}")

# Example usage:

# Show available options
show_available_options(df)

# Analyze all teams at a specific venue (sorted by matches played)
print("\n" + "="*100)
#analyze_all_teams_at_venue(df, 'Old Trafford, Manchester')

# Analyze a specific team at all venues (sorted by matches played)
print("\n" + "="*100)
#analyze_team_at_all_venues(df, 'Lancashire')

# Analyze a specific team at a specific venue
print("\n" + "="*100)
#analyze_team_performance_at_venue(df, 'Old Trafford, Manchester', 'Lancashire')


🏟️ AVAILABLE VENUES:
------------------------------
1. College Ground, Cheltenham
2. County Ground, Bristol
3. County Ground, Chelmsford
4. County Ground, Derby
5. County Ground, Hove
6. County Ground, New Road, Worcester
7. County Ground, Northampton
8. Edgbaston, Birmingham
9. Grace Road, Leicester
10. Headingley, Leeds
11. Kennington Oval, London
12. Lord's, London
13. Merchant Taylors' School Ground, Northwood
14. Old Trafford, Manchester
15. Queen's Park, Chesterfield
16. Radlett Cricket Club, Radlett
17. Riverside Ground, Chester-le-Street
18. Sophia Gardens, Cardiff
19. St Lawrence Ground, Canterbury
20. Stanley Park, Blackpool
21. The Cooper Associates County Ground, Taunton
22. The Rose Bowl, Southampton
23. Trent Bridge, Nottingham

🏏 AVAILABLE TEAMS:
------------------------------
1. Birmingham Bears
2. Derbyshire
3. Durham
4. Essex
5. Glamorgan
6. Gloucestershire
7. Hampshire
8. Kent
9. Lancashire
10. Leicestershire
11. Middlesex
12. Northamptonshire
13. Nottinghamshire
14.

In [9]:
# Analyze all teams at a specific venue (sorted by matches played)
print("\n" + "="*100)
analyze_all_teams_at_venue(df,"Edgbaston, Birmingham")

# Analyze a specific team at all venues (sorted by matches played)
print("\n" + "="*100)
#analyze_team_at_all_venues(df, 'Northamptonshire')

# Analyze a specific team at a specific venue
print("\n" + "="*100)
analyze_team_performance_at_venue(df, "Edgbaston, Birmingham", 'Derbyshire')



🏟️ COMPREHENSIVE TEAM ANALYSIS AT EDGBASTON, BIRMINGHAM

🏟️ Venue: Edgbaston, Birmingham
🏏 Team: Birmingham Bears
🗓️ Seasons: 2023, 2024
🟣 Matches played by Birmingham Bears: 16
🔥 Total Runs (1st Innings): 1836 || Avg: 183.60
🔥 Total Runs (2nd Innings): 877 || Avg: 146.17
🎯 Total Runs (Overall): 2713 || Avg: 169.56
🔵 Birmingham Bears Batted 1st Count: 10
🟢 Birmingham Bears Batted 2nd Count: 6
🏆 Wins when Batting 1st: 8 | Win %: 80.00%
🏆 Wins when Batting 2nd: 2 | Win %: 33.33%
🚀 Highest Score (HS): 228
📉 Lowest Score (LS): 99.0
✅ Highest Chase (HC): 151
⚠️ Lowest Defended (LD): 137
------------------------------------------------------------

🏟️ Venue: Edgbaston, Birmingham
🏏 Team: Derbyshire
🗓️ Seasons: 2023, 2024
🟣 Matches played by Derbyshire: 4
🔥 Total Runs (1st Innings): 179 || Avg: 179.00
🔥 Total Runs (2nd Innings): 539 || Avg: 179.67
🎯 Total Runs (Overall): 718 || Avg: 179.50
🔵 Derbyshire Batted 1st Count: 1
🟢 Derbyshire Batted 2nd Count: 3
🏆 Wins when Batting 1st: 0 | Win %: 0

,team,matches,overall_avg,win_pct_1st,win_pct_2nd,HS,LS
0,Birmingham Bears,16,169.56,80.0,33.33,228,99.0
1,Derbyshire,4,179.50,0.0,66.67,207,154.0
2,Essex,3,139.00,0.0,33.33,171,115.0
3,Worcestershire,2,181.00,100.0,0.00,187,175.0
4,Nottinghamshire,2,170.50,100.0,0.00,214,127.0
5,Durham,2,163.50,100.0,0.00,194,133.0
6,Yorkshire,2,155.50,0.0,0.00,166,145.0
7,Leicestershire,2,149.00,0.0,0.00,176,122.0
8,Somerset,2,143.50,100.0,0.00,145,142.0
9,Lancashire,2,140.50,0.0,100.00,183,98.0





🏟️ Venue: Edgbaston, Birmingham
🏏 Team: Derbyshire
🗓️ Seasons: 2023, 2024
🟣 Matches played by Derbyshire: 4
🔥 Total Runs (1st Innings): 179 || Avg: 179.00
🔥 Total Runs (2nd Innings): 539 || Avg: 179.67
🎯 Total Runs (Overall): 718 || Avg: 179.50
🔵 Derbyshire Batted 1st Count: 1
🟢 Derbyshire Batted 2nd Count: 3
🏆 Wins when Batting 1st: 0 | Win %: 0.00%
🏆 Wins when Batting 2nd: 2 | Win %: 66.67%
🚀 Highest Score (HS): 207
📉 Lowest Score (LS): 154.0
✅ Highest Chase (HC): 207
⚠️ Lowest Defended (LD): N/A


{'team': 'Derbyshire',
 'matches': 4,
 'overall_avg': 179.5,
 'win_pct_1st': 0.0,
 'win_pct_2nd': 66.67,
 'HS': 207,
 'LS': 154.0}

In [13]:
#!jupyter nbconvert --to script "Venue Analysis.ipynb" --output "Venue Analysis code.txt"
